In [1]:
#pip install flask

In [2]:
import numpy as np
import pandas as pd
import pandas as pd
from joblib import load

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer, MinMaxScaler, OneHotEncoder, OrdinalEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb


### Cargar Dataframe

In [3]:

def transformaciones(entrada):
    df =pd.DataFrame.from_dict(entrada, orient='index').T
    columnas_float = ['main_procedure', 'main_condition', 'secondary_procedure',  'asa_score',  'age', 'weight', 'height', 'anesthesia_local', 'anesthesia_sedation', 'anesthesia_plexus_block', 'anesthesia_intradural', 'anesthesia_epidural', 'anesthesia_general', 'cleaning_time' ,'recovery_time']

    df[columnas_float] = df[columnas_float].astype(float)

    #Transformacion character_of_intervention
    df['HOSP'] = df['character_of_intervention'].map(lambda x: 1 if x== 'HOSP' else 0)
    df['male'] = df['gender'].map(lambda x: 1 if x== 'male' else 0)

    df.drop(['character_of_intervention','gender'],axis=1, inplace=True)
    
    #Transformacion health_service
    df['health_service'] = df['health_service'].str.split().str[-1]
    
    #Transformacion comorbidity
    for indice, valor in df['comorbidity'].items():
        if valor in ('lung_disease', 'cancer', 'heart_disease', 'hypertension',
           'kidney_disease', 'obesity','diabetes', 'drug_abuse',
           'liver_disease', 'alcoholism', 'hiv', 'smoker'):
            df.loc[indice, 'comorbidity'] = 1
        else:
            df.loc[indice, 'comorbidity'] = 0
        
    #Transformacion asa_score
    df['asa_score'] = df['asa_score'].fillna(2.0)
    
    
    #Transformacion main_procedure
    df['main_procedure'] = df['main_procedure'].fillna(-1)

    df['main_procedure'] = df['main_procedure'].astype(str)
    df['main_procedure'] = df['main_procedure'].where(~df['main_procedure'].str.contains('[a-zA-Z]'), -1)
    df['main_procedure'] = df['main_procedure'].astype(float)
    
    def asignar_etiqueta_procedure(valor):
        diccionario_rangos_procedure = {
            (-1.01, 0.01):  'Undefined',
            (0.01, 0.96): 'Not elsewhere classified',
            (0.99, 5.9): 'Nervous System',
            (5.9, 7.99):'Endocrine System',
            (7.99, 16.99): 'Eye',
            (16.99, 17.81): 'Other MD and TP',
            (17.99, 20.99): 'Ear',
            (20.99, 29.99): 'Nose, Mouth and Pharynx',
            (29.99, 34.99): 'Respiratory System',
            (34.99, 39.99): 'Cardiovascular System',
            (39.99, 41.99): 'Hemic and Lymphatic System',
            (41.99, 54.99): 'Digestive System',
            (54.99, 59.99): 'Urinary System',
            (59.99, 64.99): 'Male Genital Organs',
            (64.99, 71.99): 'Female Genital Organs',
            (71.99, 75.99): 'Obstetrical Procedures',
            (75.99, 84.99): 'Muscoloskeletal System',
            (84.99, 86.99): 'Integumentary System',
            (86.99, 99.99): 'Miscellaneous Diagnostic'
        }
        for rango_procedure, etiqueta_procedure in diccionario_rangos_procedure.items():
            if rango_procedure[0] <= valor < rango_procedure[1]:
                return etiqueta_procedure
        return None

    df['etiqueta'] = df['main_procedure'].apply(asignar_etiqueta_procedure)

    for etiqueta_procedure in df['etiqueta'].unique():
        if etiqueta_procedure is not None:
            df[etiqueta_procedure] = df['etiqueta'].apply(lambda x: 1 if x == etiqueta_procedure else 0)

    df['etiqueta'] = df['main_procedure'].apply(asignar_etiqueta_procedure)

    etiquetas_posibles_procedure = [
        'Undefined',
        'Not elsewhere classified',
        'Nervous System',
        'Endocrine System',
        'Eye',
        'Other MD and TP',
        'Ear',
        'Nose, Mouth and Pharynx',
        'Respiratory System',
        'Cardiovascular System',
        'Hemic and Lymphatic System',
        'Digestive System',
        'Urinary System',
        'Male Genital Organs',
        'Female Genital Organs',
        'Obstetrical Procedures',
        'Muscoloskeletal System',
        'Integumentary System',
        'Miscellaneous Diagnostic'
    ]

    for etiqueta_procedure in etiquetas_posibles_procedure:
        df[etiqueta_procedure] = df['etiqueta'].apply(lambda x: 1 if x == etiqueta_procedure else 0)

    df.drop(columns=['main_procedure', 'etiqueta'], inplace=True)
            
    #Transformacion main_condition
    df['main_condition'] = df['main_condition'].fillna(-1)
   
    df['main_condition'] = df['main_condition'].astype(str)
    df['main_condition'] = df['main_condition'].str.replace(' ', '.')
    df['main_condition'] = df['main_condition'].str.rstrip('.')
    df['main_condition'] = df['main_condition'].where(~df['main_condition'].str.contains('[a-zA-Z]'), -1)
    df['main_condition'] = df['main_condition'].astype(float)
    
    def asignar_etiqueta(valor):
        diccionario_rangos = {
            (0.01, 139.8): 'INFECTIOUS AND PARASITIC DISEASES (001-139)',
            (140.0, 239.9): 'NEOPLASMS (140-239)',
            (240.0, 279.9): 'ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES, AND IMMUNITY DISORDERS (240-279)',
            (280.0, 289.9): 'DISEASES OF THE BLOOD AND BLOOD-FORMING ORGANS (280-289)',
            (290.0, 319.0): 'MENTAL, BEHAVIORAL AND NEURODEVELOPMENTAL DISORDERS (290-319)',
            (320.0, 389.9): 'DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS (320-389)',
            (390.0, 459.9): 'DISEASES OF THE CIRCULATORY SYSTEM (390-459)',
            (460.0, 519.9): 'DISEASES OF THE RESPIRATORY SYSTEM (460-519)',
            (520.0, 579.9): 'DISEASES OF THE DIGESTIVE SYSTEM (520-579)',
            (580.0, 629.9): 'DISEASES OF THE GENITOURINARY SYSTEM (580-629)',
            (630.0, 679.14): 'COMPLICATIONS OF PREGNANCY, CHILDBIRTH, AND THE PUERPERIUM (630-679)',
            (680.0, 709.9): 'DISEASES OF THE SKIN AND SUBCUTANEOUS TISSUE (680-709)',
            (710.0, 739.9): 'DISEASES OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE (710-739)',
            (740.0, 759.9): 'CONGENITAL ANOMALIES (740-759)',
            (760.0, 779.9): 'CERTAIN CONDITIONS ORIGINATING IN THE PERINATAL PERIOD (760-779)',
            (780.01, 799.9): 'SYMPTOMS, SIGNS, AND ILL-DEFINED CONDITIONS (780-799)',
            (800.0, 999.9): 'INJURY AND POISONING (800-999)',
            (-5.0, 0.0): 'Undefined',
        }
        for rango, etiqueta in diccionario_rangos.items():
            if rango[0] <= valor <= rango[1]:
                return etiqueta
        return None
    
    df['etiqueta'] = df['main_condition'].apply(asignar_etiqueta)

    for etiqueta in df['etiqueta'].unique():
        if etiqueta is not None:
            df[etiqueta] = df['etiqueta'].apply(lambda x: 1 if x == etiqueta else 0)
            
    
    df['etiqueta'] = df['main_condition'].apply(asignar_etiqueta)


    etiquetas_posibles = [
        'INFECTIOUS AND PARASITIC DISEASES (001-139)',
        'NEOPLASMS (140-239)',
        'ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES, AND IMMUNITY DISORDERS (240-279)',
        'DISEASES OF THE BLOOD AND BLOOD-FORMING ORGANS (280-289)',
        'MENTAL, BEHAVIORAL AND NEURODEVELOPMENTAL DISORDERS (290-319)',
        'DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS (320-389)',
        'DISEASES OF THE CIRCULATORY SYSTEM (390-459)',
        'DISEASES OF THE RESPIRATORY SYSTEM (460-519)',
        'DISEASES OF THE DIGESTIVE SYSTEM (520-579)',
        'DISEASES OF THE GENITOURINARY SYSTEM (580-629)',
        'COMPLICATIONS OF PREGNANCY, CHILDBIRTH, AND THE PUERPERIUM (630-679)',
        'DISEASES OF THE SKIN AND SUBCUTANEOUS TISSUE (680-709)',
        'DISEASES OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE (710-739)',
        'CONGENITAL ANOMALIES (740-759)',
        'CERTAIN CONDITIONS ORIGINATING IN THE PERINATAL PERIOD (760-779)',
        'SYMPTOMS, SIGNS, AND ILL-DEFINED CONDITIONS (780-799)',
        'INJURY AND POISONING (800-999)',
        'Undefined',
        ]

    for etiqueta in etiquetas_posibles:
        df[etiqueta] = df['etiqueta'].apply(lambda x: 1 if x == etiqueta else 0)

    df.drop(columns=['main_condition', 'etiqueta'], inplace=True)
    
    df['health_service'] = df['health_service'].astype(int)
    df['comorbidity'] = df['comorbidity'].astype(int)   

    df.loc[:,['secondary_procedure', 'health_service', 'asa_score', 'age', 'weight',
       'height', 'comorbidity', 'anesthesia_local', 'anesthesia_sedation',
       'anesthesia_plexus_block', 'anesthesia_intradural',
       'anesthesia_epidural', 'anesthesia_general', 'cleaning_time', 'recovery_time',
       'HOSP', 'male', 'Respiratory System', 'Eye',
       'Female Genital Organs', 'Other MD and TP', 'Digestive System',
       'Undefined', 'Urinary System', 'Muscoloskeletal System',
       'Hemic and Lymphatic System', 'Integumentary System',
       'Male Genital Organs', 'Nose, Mouth and Pharynx',
       'Cardiovascular System', 'Nervous System', 'Ear',
       'Not elsewhere classified', 'Obstetrical Procedures',
       'Endocrine System', 'Miscellaneous Diagnostic', 'NEOPLASMS (140-239)',
       'DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS (320-389)',
       'DISEASES OF THE DIGESTIVE SYSTEM (520-579)',
       'DISEASES OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE (710-739)',
       'INFECTIOUS AND PARASITIC DISEASES (001-139)',
       'DISEASES OF THE GENITOURINARY SYSTEM (580-629)',
       'INJURY AND POISONING (800-999)',
       'DISEASES OF THE RESPIRATORY SYSTEM (460-519)',
       'SYMPTOMS, SIGNS, AND ILL-DEFINED CONDITIONS (780-799)',
       'DISEASES OF THE SKIN AND SUBCUTANEOUS TISSUE (680-709)',
       'DISEASES OF THE CIRCULATORY SYSTEM (390-459)',
       'ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES, AND IMMUNITY DISORDERS (240-279)',
       'COMPLICATIONS OF PREGNANCY, CHILDBIRTH, AND THE PUERPERIUM (630-679)',
       'CONGENITAL ANOMALIES (740-759)',
       'MENTAL, BEHAVIORAL AND NEURODEVELOPMENTAL DISORDERS (290-319)',
       'DISEASES OF THE BLOOD AND BLOOD-FORMING ORGANS (280-289)',
       'CERTAIN CONDITIONS ORIGINATING IN THE PERINATAL PERIOD (760-779)']]

    return df


### Transformaciones a los datos que llegan 

# Construccion API

In [4]:
from flask import Flask, request, jsonify
import json
import joblib
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [5]:
app = Flask(__name__)

In [6]:
#Se carga el pipeline
model = joblib.load('mejor_modelo_1.joblib')

In [7]:
importances = model.feature_importances_

In [8]:
importances

array([0.00969146, 0.01466466, 0.00494271, 0.00605896, 0.00148712,
       0.00143487, 0.00161801, 0.01266215, 0.00690033, 0.014665  ,
       0.01820096, 0.04188753, 0.11878247, 0.0014269 , 0.00855425,
       0.36048877, 0.00456384, 0.00367684, 0.0026238 , 0.02620774,
       0.02307064, 0.00504154, 0.00754294, 0.01037978, 0.01177106,
       0.00172155, 0.01291241, 0.00553998, 0.00636168, 0.03073322,
       0.01258285, 0.02073552, 0.01186428, 0.        , 0.00959842,
       0.00082086, 0.01902276, 0.0123312 , 0.05329906, 0.01347162,
       0.00287375, 0.01035515, 0.00416947, 0.02823772, 0.00184668,
       0.00228172, 0.01103302, 0.00524533, 0.        , 0.00326157,
       0.00071905, 0.0006368 , 0.        ], dtype=float32)

In [9]:
#Se define la ruta
@app.route('/nuevaPrediccion', methods=['POST'])
def prediccion():
    data = request.get_json(force=True)
    df = transformaciones(data)
    input = df.values.reshape(1,-1)
    print(input)
    pred = model.predict(input)
    list = pred.tolist()
    return json.dumps(list)

In [10]:
#Se levanta la aplicación en el puerto 8080 para escuchar nuevas predicciones
if __name__ == '__main__':
    app.run(port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [09/May/2024 19:39:06] "POST /nuevaPrediccion HTTP/1.1" 200 -


[[  0.   8.   3.  57. 111. 188.   1.   0.   0.   0.   1.   1.   0.  24.
   65.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
